In [57]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler

import time


In [58]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [59]:
df = pd.read_csv('/Users/freekcool/Documents/GitHub/dmt-assignment-2/shreyas/train_preprocessed.csv')[:123959]
df.head()

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,random_bool,click_bool,booking_bool,month,relevance,prop_starrating_location_interaction,price_historical_interaction,total_time_duration,highly_rated_prop,total_people
0,1,0,0,1,893,3,3.5,1,2.83,0.0438,...,1,0,0,4,0,8.49,4.949896,0,0,4
1,1,0,0,1,10404,4,4.0,1,2.20,0.0149,...,1,0,0,4,0,8.80,5.029827,0,1,4
2,1,0,0,1,21315,3,4.5,1,2.20,0.0245,...,1,0,0,4,0,6.60,4.919822,0,0,4
3,1,0,0,1,27348,2,4.0,1,2.83,0.0125,...,1,0,0,4,0,5.66,4.389468,0,0,4
4,1,0,0,1,29604,4,3.5,1,2.64,0.1241,...,1,0,0,4,0,10.56,4.929858,0,1,4


In [60]:
X = df
train_groups = X.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

relevances = X['relevance']

X= X.loc[:, ~X.columns.isin(['srch_id','relevance'])]
y = relevances

In [61]:
X.columns

Index(['site_id', 'visitor_location_country_id', 'prop_country_id', 'prop_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool',
       'click_bool', 'booking_bool', 'month',
       'prop_starrating_location_interaction', 'price_historical_interaction',
       'total_time_duration', 'highly_rated_prop', 'total_people'],
      dtype='object')

In [62]:
X = X.drop(['booking_bool','position','random_bool','prop_id','click_bool'], axis=1)


In [63]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [64]:
y_mapped = y.replace({0: 0, 1: 1, 5: 2})

In [65]:
y = y.values

In [66]:
# Check unique values in y
unique_values = pd.unique(y)
print(unique_values)

[0 5 1]


## Create a model

In [67]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [68]:
y_mapped = np.where(y==5, 2, y)

In [69]:
y_one_hot = tf.keras.utils.to_categorical(y_mapped)

In [70]:
# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Assuming 3 classes (0, 1, 5)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [71]:
model.fit(X, y_one_hot, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
3099/3099 [==============================] - 32s 10ms/step - loss: 5.3813 - accuracy: 0.9196 - val_loss: 1.2460 - val_accuracy: 0.9554
Epoch 2/100
3099/3099 [==============================] - 33s 10ms/step - loss: 0.7255 - accuracy: 0.9411 - val_loss: 0.3362 - val_accuracy: 0.9554
Epoch 3/100
3099/3099 [==============================] - 32s 10ms/step - loss: 0.2772 - accuracy: 0.9521 - val_loss: 0.2146 - val_accuracy: 0.9554
Epoch 4/100
3099/3099 [==============================] - 33s 10ms/step - loss: 0.2157 - accuracy: 0.9549 - val_loss: 0.2119 - val_accuracy: 0.9554
Epoch 5/100
3099/3099 [==============================] - 32s 10ms/step - loss: 0.2139 - accuracy: 0.9550 - val_loss: 0.2120 - val_accuracy: 0.9554
Epoch 6/100
3099/3099 [==============================] - 32s 10ms/step - loss: 0.2149 - accuracy: 0.9551 - val_loss: 0.2121 - val_accuracy: 0.9554
Epoch 7/100
3099/3099 [==============================] - 33s 11ms/step - loss: 0.2151 - accuracy: 0.9550 - val_loss: 0

3099/3099 [==============================] - 33s 11ms/step - loss: 0.2131 - accuracy: 0.9551 - val_loss: 0.2123 - val_accuracy: 0.9554
Epoch 57/100
3099/3099 [==============================] - 33s 11ms/step - loss: 0.2131 - accuracy: 0.9551 - val_loss: 0.2124 - val_accuracy: 0.9554
Epoch 58/100
3099/3099 [==============================] - 33s 11ms/step - loss: 0.2132 - accuracy: 0.9551 - val_loss: 0.2126 - val_accuracy: 0.9554
Epoch 59/100
3099/3099 [==============================] - 33s 11ms/step - loss: 0.2132 - accuracy: 0.9551 - val_loss: 0.2125 - val_accuracy: 0.9554
Epoch 60/100
3099/3099 [==============================] - 33s 11ms/step - loss: 0.2140 - accuracy: 0.9551 - val_loss: 0.2123 - val_accuracy: 0.9554
Epoch 61/100
3099/3099 [==============================] - 33s 11ms/step - loss: 0.2130 - accuracy: 0.9551 - val_loss: 0.2123 - val_accuracy: 0.9554
Epoch 62/100
3099/3099 [==============================] - 33s 11ms/step - loss: 0.2134 - accuracy: 0.9551 - val_loss: 0.2121 

In [72]:
model.save('/Users/freekcool/Documents/dmt-neural.h5')

## Predicting on the testset

In [53]:
test_df = pd.read_csv('/Users/freekcool/Documents/GitHub/dmt-assignment-2/shreyas/test_preprocessed.csv')

In [54]:
X.columns

Index(['site_id', 'visitor_location_country_id', 'prop_country_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'month',
       'prop_starrating_location_interaction', 'price_historical_interaction',
       'total_time_duration', 'highly_rated_prop', 'total_people'],
      dtype='object')

In [55]:
test_df.columns

Index(['srch_id', 'site_id', 'visitor_location_country_id', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'orig_destination_distance', 'month',
       'prop_starrating_location_interaction', 'price_historical_interaction',
       'total_time_duration', 'highly_rated_prop', 'total_people'],
      dtype='object')

In [56]:
# Select relevant features and make sure to remove any features that aren't in your training set.
test_X = test_df.loc[:, ~test_df.columns.isin(['booking_bool','position','random_bool','prop_id','click_bool'])]

# Scale the test data using the same scaler used for training data.
test_X_scaled = scaler.transform(test_X)

# Make predictions
predictions = model.predict(test_X_scaled)

/Users/freekcool/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- srch_id
Feature names seen at fit time, yet now missing:
- booking_bool
- click_bool
- position
- prop_id
- random_bool

  warnings.warn(message, FutureWarning)


ValueError: X has 26 features, but MinMaxScaler is expecting 30 features as input.